In [22]:
import pandas as pd

df = pd.read_csv('/content/train.csv',engine='python',error_bad_lines=False)

<ipython-input-22-5e3ef21fae71>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/train.csv',engine='python',error_bad_lines=False)
Skipping line 2394: unexpected end of data


In [23]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
df.isnull().sum()

id          0
title      64
author    244
text        6
label       0
dtype: int64

In [25]:
df.shape

(2392, 5)

In [26]:
df = df.dropna()

In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [27]:
df.shape

(2084, 5)

In [28]:
X = df.drop('label',axis=1)
y=df['label']

In [29]:
## Check whether dataset is balanced or not
y.value_counts()

0    1173
1     911
Name: label, dtype: int64

In [30]:
import tensorflow as tf


In [31]:
tf.__version__

'2.12.0'

In [34]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [35]:
## vocabulary size
voc_size = 5000

In [36]:
messages = X.copy()

In [37]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
2387,2387,"Statue of Marx, Funded by China, Will Stand in...",Melissa Eddy,BERLIN — The bicentennial of the birth of K...
2388,2388,A Curious Plan to Fight Climate Change: Buy Mi...,Michael Corkery and Michael Wines,"FAIRVIEW, W. Va. — The coal was piled about..."
2389,2389,Study: Two Hours of Social Media Use a Day Dou...,Charlie Nash,Just two hours of social media use a day can d...
2390,2390,Skepticism and Support in South Korea as Ban K...,Choe Sang-Hun,"HAENGCHI VILLAGE, South Korea — Each day hu..."


In [39]:
import nltk
import re
from nltk.corpus import stopwords

In [40]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
messages.reset_index(inplace=True)

In [44]:
## Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [46]:
onehot_rep = [one_hot(word,voc_size) for word in corpus]


In [47]:
onehot_rep

[[1984, 3069, 1424, 4172, 3419, 4666, 400, 2704, 4864, 339],
 [2302, 369, 2951, 2621, 1303, 1367, 3207],
 [2848, 3823, 2648, 1806],
 [707, 4025, 2517, 4579, 3735, 1728],
 [371, 1303, 4002, 4258, 640, 3468, 1303, 1710, 1988, 441],
 [2465,
  3362,
  2234,
  2617,
  1241,
  4167,
  785,
  2930,
  1934,
  274,
  4152,
  2193,
  4274,
  2362,
  3207],
 [856, 2906, 303, 3777, 495, 1692, 4139, 3903, 2775, 187, 2822],
 [1713, 1173, 3376, 2463, 1660, 3095, 4167, 77, 2775, 187, 2822],
 [1627, 1754, 2630, 101, 3219, 440, 240, 2167, 4167, 3397],
 [3851, 3142, 665, 889, 4478, 2303, 1176, 792],
 [1693, 4718, 2098, 389, 4080, 251, 1570, 207, 4097, 1203, 2171],
 [4579, 4252, 3419, 440, 4167, 1660],
 [3365, 1566, 3726, 3530, 1711, 4325, 4721, 4610, 2038],
 [109, 1113, 1240, 4516, 4232, 59, 746, 2775, 187, 2822],
 [2437, 3431, 461, 4807, 3607, 2775, 187, 2822],
 [140, 3679, 1295, 1, 1782, 877, 3585, 2485, 227, 3111],
 [1085, 546, 369],
 [802, 1266, 4208, 4101, 4167, 1630, 3183, 3207],
 [3859, 3110, 2951

In [49]:
## Apply Embedding representation

sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 2704, 4864,  339],
       [   0,    0,    0, ..., 1303, 1367, 3207],
       [   0,    0,    0, ..., 3823, 2648, 1806],
       ...,
       [   0,    0,    0, ..., 1711,  146, 3207],
       [   0,    0,    0, ..., 2775,  187, 2822],
       [   0,    0,    0, ..., 3436, 1732, 3172]], dtype=int32)

In [50]:
## Creating Model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [52]:
## Train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [53]:
## Model training

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
44/44 [==============================] - 17s 164ms/step - loss: 0.5919 - accuracy: 0.7206 - val_loss: 0.3258 - val_accuracy: 0.8634
Epoch 2/10
44/44 [==============================] - 3s 73ms/step - loss: 0.1554 - accuracy: 0.9405 - val_loss: 0.3038 - val_accuracy: 0.8808
Epoch 3/10
44/44 [==============================] - 2s 38ms/step - loss: 0.0385 - accuracy: 0.9914 - val_loss: 0.3587 - val_accuracy: 0.8968
Epoch 4/10
44/44 [==============================] - 1s 33ms/step - loss: 0.0100 - accuracy: 0.9986 - val_loss: 0.4722 - val_accuracy: 0.8924
Epoch 5/10
44/44 [==============================] - 1s 28ms/step - loss: 0.0057 - accuracy: 0.9993 - val_loss: 0.5318 - val_accuracy: 0.8895
Epoch 6/10
44/44 [==============================] - 1s 21ms/step - loss: 0.0055 - accuracy: 0.9993 - val_loss: 0.5318 - val_accuracy: 0.8953
Epoch 7/10
44/44 [==============================] - 1s 34ms/step - loss: 0.0046 - accuracy: 0.9993 - val_loss: 0.5977 - val_accuracy: 0.8808
Epoch 8/10


In [54]:
y_pred = model.predict(X_test)

22/22 [==============================] - 1s 3ms/step


In [59]:
y_pred = np.where(y_pred >= 0.5,1,0)

In [60]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[362,  29],
       [ 59, 238]])

In [61]:
accuracy_score(y_test,y_pred)

0.872093023255814